## What is Synthetic Control Method?

I will try to keep this part short and focus more on why Data scientists should care about such methods and how to use them on larger datasets based on practical experience using  [SparseSC package](https://github.com/microsoft/SparseSC).

The Synthetic Control (SC) method is a statistical method used to estimate causal effects from binary treatments on observational panel (longitudinal) data. The method got quite a coverage by being described as [“the most important innovation in the policy evaluation literature in the last few years”](https://www.aeaweb.org/articles?id=10.1257/jep.31.2.3) and got an article published in [Washington Post - Seriously, here’s one amazing math trick to learn what can’t be known](https://www.washingtonpost.com/news/wonk/wp/2015/10/30/how-to-measure-things-in-a-world-of-competing-claims/). “SC is a technique to create an artificial control group by taking a weighted average of untreated units in such a way that it reproduces the characteristics of the treated units before the intervention(treatment). The SC acts as the counterfactual for a treatment unit, and the estimate of a treatment effect is the difference between the observed outcome in the post-treatment period and the SC's outcome.”

“One way to think of SC is as an improvement upon [difference-in-difference (DiD) estimation](https://en.wikipedia.org/wiki/Difference_in_differences). Typical DiD will compare a treated unit to the average of the control units. But often the treated unit does not look like a typical control (e.g., it might have a different growth rate), in which case the 'parallel trend' assumption of DiD is not valid. SC remedies this by choosing a smarter linear combination, rather than the simple average, to weigh more heavily the more similar units. SC's assumption is if there are endogenous factors that affect treatment and future outcomes then you should be able to control them by matching past outcomes. The matching that SC provides can therefore deal with some problems in estimation that DiD cannot handle.”

Here is the link to the Causal inference book which I found most useful to understand the math behind SC- [Causal Inference for The Brave and True by Matheus Facure - Chapter 15](https://matheusfacure.github.io/python-causality-handbook/15-Synthetic-Control.html).

## Why should any Data scientist care about this method?

Often as a Data Scientist, you will encounter situations as follows where running A/B testing is not feasible  because of -

1. Lack of infrastructure
2. Lack of similar groups for running A/B testing (in case of evaluation of state policies, as there is no state equivalent of other) 
3. Providing unwanted advantage to one group over others. Sometimes running an A/B test can give an unfair advantage and lead you into anti-trust territory. For example, what if Amazon tries to charge differential pricing for different customers or apply different margins for their sellers for the same product?

As a data scientist, stakeholders may still ask you to estimate the impact of certain changes/treatments, and Synthetic controls can come to the rescue in this situation. For this reason, it is a valuable tool to keep in your algorithmic toolkit.

## Problem Overview

We will use the Proposition 99 data to explain the use case for this approach and also how to use the SparceSC library and its key features. “In 1988, California passed a famous Tobacco Tax and Health Protection Act, which became known as Proposition 99. Its primary effect is to impose a 25-cent per pack state excise tax on the sale of tobacco cigarettes within California, with approximately equivalent excise taxes similarly imposed on the retail sale of other commercial tobacco products, such as cigars and chewing tobacco. Additional restrictions placed on the sale of tobacco include a ban on cigarette vending machines in public areas accessible by juveniles, and a ban on the individual sale of single cigarettes. Revenue generated by the act was earmarked for various environmental and health care programs, and anti-tobacco advertisements. To evaluate its effect, we can gather data on cigarette sales from multiple states and across a number of years. In our case, we got data from the year 1970 to 2000 from 39 states.”

In [ ]:
import os
install = '"git+https://github.com/microsoft/SparseSC.git"'
os.system(f'pip install -Uqq {install}')

In [1]:
import pandas as pd
import numpy as np
import SparseSC
from datetime import datetime
import warnings
import plotly.express as px
import plotly.graph_objects as pgo
pd.set_option("display.max_columns", None)
warnings.filterwarnings('ignore')

Let's look at the data

We have data per `state` as treatment unit and yearly (`year` column) per-capita sales of cigarettes in packs (`cigsale` column) and the cigarette retail price (`retprice` column). We are going to pivot this data so that each row is one treatment unit(`state`), and columns represent the yearly `cigsale` value.

In [ ]:
%%capture
!pip install gdown --no-cache-dir

In [3]:
!gdown 1tK9L_ouFtMM8unUIEp5dQMPjmBHP8HdB # grp
!gdown 1pYRWuWQxDu1qT-kj4T2GMsmQ0k-fMqZz # industry
!gdown 1SggyYY5gmZo1hmG3sYJqCnr1iP5oYDzi # export

"gdown" no se reconoce como un comando interno o externo,
programa o archivo por lotes ejecutable.
"gdown" no se reconoce como un comando interno o externo,
programa o archivo por lotes ejecutable.
"gdown" no se reconoce como un comando interno o externo,
programa o archivo por lotes ejecutable.


In [2]:
df_grp = pd.read_csv('/kaggle/working/br_grp.csv')
df_industry = pd.read_csv('/kaggle/working/br_industry.csv')
df_export = pd.read_csv('/kaggle/working/br_export.csv')

FileNotFoundError: [Errno 2] No such file or directory: '/kaggle/working/br_grp.csv'

In [ ]:
print(df_grp.isna().sum().sum())
print(df_industry.isna().sum().sum())
print(df_export.isna().sum().sum())

In [ ]:
df_export.drop(columns='Unnamed: 0', inplace=True)
df_export.columns = df_export.iloc[0, :].values
df_export.drop(0, inplace=True)
df_export.rename(columns={'Date': 'region'}, inplace=True)

In [ ]:
df_grp['region'] = df_grp['region'].str.rstrip().str.lower()
df_industry['region'] = df_industry['region'].str.rstrip().str.lower()
df_export['region'] = df_export['region'].str.rstrip().str.lower()

In [ ]:
for rname in LIST_OF_REGIONS_WITH_SEZS:
    for df in [df_grp, df_industry, df_export]:
        if rname not in df['region'].tolist():
            print('rname is not present')
print('finished')

In [ ]:
df_grp.index = df_grp['region']
df_grp.drop(columns='region', inplace=True)
df_industry.index = df_industry['region']
df_industry.drop(columns='region', inplace=True)
df_export.index = df_export['region']
df_export.drop(columns='region', inplace=True)

In [ ]:
df_grp.head(3)

In [ ]:
df_industry.head(3)

In [ ]:
df_export.head(3)

In [ ]:
df_grp.columns = df_grp.columns.astype(int)
df_industry.columns = df_industry.columns.astype(int)
df_export.columns = df_export.columns.astype(int)

In [ ]:
sezs_region_and_year = [
    ('ceará', 2010),
    ('piauí', 2010),
    ('acre', 2010),
    ('pernambuco', 2014),
    ('minas gerais', 2018)
]

In [ ]:
LIST_OF_REGIONS_WITH_SEZS = [x[0] for x in sezs_region_and_year]

In [ ]:
DATAFRAMES = {
    'grp': df_grp, 
    'industry': df_industry, 
    'export': df_export, 
}

In [ ]:
from tqdm.notebook import tqdm

In [ ]:
def calculate_synths(current_region_name, current_year):
    
    for rname in LIST_OF_REGIONS_WITH_SEZS:
        if rname != current_region_name:
            idx_to_drop = [val for val in list(DATAFRAMES.values())[0].index.values if val == rname]
    
    synths = {}
    for df_name, df in tqdm(DATAFRAMES.items()):
        synth = SparseSC.fit(
            features=df.drop(idx_to_drop, axis=0).iloc[:,df.columns <= current_year].values,
            targets=df.drop(idx_to_drop).iloc[:,df.columns > current_year].values,
            treated_units=[idx for idx, val in enumerate(df.index.values) if val == current_region_name],
            progress=0, print_path=False
        )
        treated_units=[idx for idx, val in enumerate(df.index.values) if val == current_region_name]
        result = df.loc[df.index == current_region_name].T.reset_index(drop=False)
        result.columns = ["year", "Observed"]
        result['Synthetic'] = synth.predict(df.drop(idx_to_drop, axis=0).values)[treated_units,:][0]
        synths[df_name] = {'synth': synth, 'res_df': result}
    return synths

In [ ]:
synths_for_each_region = {}
for current_region, current_year in sezs_region_and_year:
    print(f"Calculating for {current_region}")
    synths_for_each_region[current_region] = calculate_synths(current_region, current_year)


In [ ]:
def vizualize(result: pd.DataFrame, region: str, variable: str, year: int):
    fig = px.line(
            data_frame = result, 
            x = "year", 
            y = ["Observed","Synthetic"], 
            template = "plotly_dark",)

    fig.add_trace(
        pgo.Scatter(
            x=[year,year],
            y=[0, result.Observed.max()*1.02 if result.Observed.max() > result.Synthetic.max() else result.Synthetic.max()*1.02],
            #y=[result.Observed.min()*0.98,result.Observed.max()*1.02], 
            line={
                'dash': 'dash',
            }, name='SEZ creation'
        ))
    fig.update_layout(
            title  = {
                'text':f"Synthetic Control Assessment for {region}",
                'y':0.95,
                'x':0.5,
            },
            legend =  dict(y=1, x= 0.1, orientation='v'),
            legend_title = "",
            xaxis_title="Year", 
            yaxis_title=variable,
            font = dict(size=15)
    )
    fig.show(renderer='notebook')

In [ ]:
print(sezs_region_and_year)

grp, industry, export

In [ ]:
synths_for_each_region['ceará']['grp']['res_df'][synths_for_each_region['ceará']['grp']['res_df']['year'] > 1995]

In [ ]:
vizualize(synths_for_each_region['acre']['industry']['res_df'], 'Acre state', 'Industry production', 2010)

In [ ]:
pd.set_option('display.float_format', lambda x: '%.3f' % x)

In [ ]:
from sklearn.metrics import mean_absolute_percentage_error as MAPE
#from sklearn.metrics import mean_absolute_error as MAE

In [ ]:
print(sezs_region_and_year)

In [ ]:
df = synths_for_each_region['minas gerais']['export']['res_df']
treatment_year = 2018
mape_pre = MAPE(df[df['year'] <= treatment_year]['Observed'].values, df[df['year'] <= treatment_year]['Synthetic'].values)
mape_post = MAPE(df[df['year'] > treatment_year]['Observed'].values, df[df['year'] > treatment_year]['Synthetic'].values)
pd.DataFrame({
    'Pre': [mape_pre],
    'Post': [mape_post],
    'Post/Pre': [mape_post/mape_pre]
}, index=['APE'])

In [ ]:
df_export.columns

In [ ]:
with open('bullshit_for_brazil.txt', 'w') as f: 
    
    for rname, year in sezs_region_and_year:
        print(rname, year)
        f.write(rname)
        f.write('\n')
        for variable in tqdm(synths_for_each_region[rname].keys()):
            f.write(variable)
            f.write('\n')
            if variable == 'grp':
                colnames = np.arange(1989, 2021)
            elif variable == 'industry':
                colnames = np.arange(2002, 2021)
            elif variable == 'export':
                colnames = np.arange(1989, 2021)
            else:
                raise RuntimeError 
            df = pd.DataFrame(np.hstack((
                synths_for_each_region[rname][variable]['synth'].features, 
                synths_for_each_region[rname][variable]['synth'].targets)), columns=colnames)
            year = year
            ## Creating unit treatment_periods
            unit_treatment_periods = np.full((df.values.shape[0]), np.nan)
            unit_treatment_periods[synths_for_each_region[rname][variable]['synth'].treated_units] = [idx for idx, colname in enumerate(df.columns) if colname == year][0]

            ## fitting estimate effects method
            sc = SparseSC.estimate_effects(
                outcomes = df.values,  
                unit_treatment_periods = unit_treatment_periods, 
                max_n_pl=50, # Number of placebos
                level=0.9 # Level for confidence intervals
            )
            f.write(str(sc))
            f.write('\n\n')
            f.write(f"Estimated effect of SEZ is {np.round(sc.pl_res_post.effect_vec.effect[-1])}, \
                    with a p-value of  {np.round(sc.pl_res_post.effect_vec.p[-1],2)}")            
            f.write('\n\n')

In [ ]:
print(f"Estimated effect of SEZ is {np.round(sc.pl_res_post.effect_vec.effect[-1])}, \
with a p-value of  {np.round(sc.pl_res_post.effect_vec.p[-1],2)}")

In [ ]:
synths_for_each_region['oryol region']['fixed_assets']['synth'].show()